**Generating Task associated models**
- Developed by Francisco de Asís Guil Asensio

In [41]:
import cobra
import time
import numpy as np
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import json
import pickle
import openpyxl
import os
from pathlib import Path

In [42]:
cobra.Configuration().solver="cplex"

Lee listas de nombres de modelos

In [43]:
base=os.listdir("/home/alumno15/modelos/base")
tejidos=os.listdir("/home/alumno15/modelos/tejidos")
MM=os.listdir("/home/alumno15/modelos/MM")

Lee hoja con tareas metabólicas

In [44]:
excel = openpyxl.load_workbook('/home/alumno15/tasks.xlsx')
hoja=excel['TASKS']

In [45]:
def lineaBlanco(i):
    for j in range(1,14):
        if not hoja.cell(row=i,column=j).value==None:
            return(False)
    return(True)

In [46]:
def describeTarea(i0,i1):
    
    entradas=[]
    compartimento_entradas=[]
    LB_entradas=[]
    UB_entradas=[]
    
    salidas=[]
    compartimento_salidas=[]
    LB_salidas=[]
    UB_salidas=[]
    
    entradas_ecuacion=[]
    salidas_ecuacion=[]
    LB_ecuacion=[]
    UB_ecuacion=[]
    
    print("Entradas")
    for i in range(i0,i1+1):
        ins=hoja.cell(row=i,column=5).value
        LB=hoja.cell(row=i,column=6).value
        UB=hoja.cell(row=i,column=7).value
        if LB==None:
            LB=0
        if UB==None:
            UB=1000
        
        LLB=LB
        LB=-UB
        UB=-LLB
            
            
        if not ins==None:
            metas=ins.split(";")
            for name in metas:
                nombre=name[:len(name)-3]
                compartimento=name[len(name)-3:]
                print(nombre,compartimento,LB,UB)
                entradas.append(nombre)
                compartimento_entradas.append(compartimento)
                LB_entradas.append(LB)
                UB_entradas.append(UB)

            
                
    print("Salidas")
    for i in range(i0,i1+1):
        outs=hoja.cell(row=i,column=8).value
        LB=hoja.cell(row=i,column=9).value
        UB=hoja.cell(row=i,column=10).value
        if LB==None:
            LB=0
        if UB==None:
            UB=1000
        if not outs==None:
            metas=outs.split(";")
            for name in metas:
                nombre=name[:len(name)-3]
                compartimento=name[len(name)-3:]
                print(nombre,compartimento,LB,UB)
                salidas.append(nombre)
                compartimento_salidas.append(compartimento)
                LB_salidas.append(LB)
                UB_salidas.append(UB)
    
    for i in range(i0,i1+1):
        ecu=hoja.cell(row=i,column=11).value
        if not ecu==None:
            if " => " in ecu:
                LB=hoja.cell(row=i,column=12).value
                if LB==None:
                    LB=0
                UB=hoja.cell(row=i,column=13).value
                if UB==None:
                    UB=1000
                partes=ecu.split(" => ")
                
                entradas_ecuacion.append(partes[0].split(" + "))
                salidas_ecuacion.append(partes[1].split(" + "))
                LB_ecuacion.append(LB)
                UB_ecuacion.append(UB)
                
                print("Ecuacion")
                print("Entradas",entradas_ecuacion,"Salidas",salidas_ecuacion)
                print("LB =",LB_ecuacion," UB =",UB_ecuacion)
                
                
            if " <=> " in ecu:
                LB=hoja.cell(row=i,column=12).value
                if LB==None:
                    LB=-1000
                UB=hoja.cell(row=i,column=13).value
                if UB==None:
                    UB=1000
                partes=ecu.split(" <=> ")
                entradas_ecuacion.append(partes[0].split(" + "))
                salidas_ecuacion.append(partes[1].split(" + "))
                LB_ecuacion.append(LB)
                UB_ecuacion.append(UB)
                
                print("Ecuacion")
                print("Entradas",entradas_ecuacion,"Salidas",salidas_ecuacion)
                print("LB =",LB_ecuacion," UB =",UB_ecuacion)
                

            
    
    print()
    
    return([entradas,compartimento_entradas,LB_entradas,UB_entradas,salidas,compartimento_salidas,LB_salidas,UB_salidas,entradas_ecuacion,
           salidas_ecuacion,LB_ecuacion,UB_ecuacion])

In [47]:
Task=1
inTask=False
inicios=[]
finales=[]

for i in range(2,hoja.max_row+1):
    if  not lineaBlanco(i) and not hoja.cell(row=i,column=1).value=="#":
        if not hoja.cell(row=i,column=3).value==None:
            if not inTask:
                inTask=True
            else:
                Task+=1
                finales.append(i-1)
            ini=i
            inicios.append(i)
        
    
    else:
        if inTask:
            finales.append(i-1)
            Task+=1
            inTask=False
finales.append(i)

task=[]
for i in range(len(inicios)):
    #print("Task",i,"empieza en linea",inicios[i],"y acaba en linea",finales[i])
    task.append(describeTarea(inicios[i],finales[i]))

Entradas
O2 [s] -1000 0
glucose [s] -1000 0
Salidas
H2O [s] 0 1000
CO2 [s] 0 1000
Ecuacion
Entradas [['ATP[c]', 'H2O[c]']] Salidas [['ADP[c]', 'Pi[c]', 'H+[c]']]
LB = [1]  UB = [1000]

Entradas
O2 [s] -1000 0
glucose [s] -1000 0
Salidas
H2O [s] 0 1000
CO2 [s] 0 1000
Ecuacion
Entradas [['GTP[c]', 'H2O[c]']] Salidas [['GDP[c]', 'Pi[c]', 'H+[c]']]
LB = [1]  UB = [1000]

Entradas
O2 [s] -1000 0
glucose [s] -1000 0
Salidas
H2O [s] 0 1000
CO2 [s] 0 1000
Ecuacion
Entradas [['CTP[c]', 'H2O[c]']] Salidas [['CDP[c]', 'Pi[c]', 'H+[c]']]
LB = [1]  UB = [1000]

Entradas
O2 [s] -1000 0
glucose [s] -1000 0
Salidas
H2O [s] 0 1000
CO2 [s] 0 1000
Ecuacion
Entradas [['UTP[c]', 'H2O[c]']] Salidas [['UDP[c]', 'Pi[c]', 'H+[c]']]
LB = [1]  UB = [1000]

Entradas
O2 [s] -1000 0
glucose [s] -1000 0
NH3 [s] -1000 0
Pi [s] -1000 0
Salidas
H2O [s] 0 1000
CO2 [s] 0 1000
ATP [c] 1 1000
Ecuacion
Entradas [['ATP[c]', 'H2O[c]']] Salidas [['ADP[c]', 'Pi[c]', 'H+[c]']]
LB = [-1000]  UB = [1000]

Entradas
O2 [s] -1000 0
g

In [48]:
def buscaMetabolitoMat(modelo,nombre,compartment):
    extension=""
    if compartment=="[s]":
        extension="e"
    if compartment=="[m]":
        extension="m"
    if compartment=="[x]":
        extension="e"
    if compartment=="[c]":
        extension="c"
        
    identifier=""
    for m in modelo.metabolites:
        if m.name==nombre and m.id[len(m.id)-1]==extension:
            identifier=m.id
    try:
        meta=modelo.metabolites.get_by_id(identifier)
    except:
        meta=0
    if meta==0:
        print("Fallo")
    return(meta)

In [49]:
def buscaMetabolitoSBML(modelo,nombre,compartment):
    extension=""
    if compartment=="[s]":
        compartment="e"
    if compartment=="[m]":
        compartment="m"
    if compartment=="[x]":
        compartment="e"
    if compartment=="[c]":
        compartment="c"
    
    
    identifier=""
    for m in modelo.metabolites:
        if m.name.split(" [")[0]==nombre and m.compartment==compartment:
            identifier=m.id
    try:
        meta=modelo.metabolites.get_by_id(identifier)
    except:
        meta=0
    if meta==0:
        print("Fallo 2")

    return(meta)

In [50]:
def creaModeloTarea(modelo,k,verbose):
    with modelo:
        miTask=task[k]
        entradas=miTask[0]
        compartimento_entradas=miTask[1]
        LB_entradas=miTask[2]
        UB_entradas=miTask[3]

        salidas=miTask[4]
        compartimento_salidas=miTask[5]
        LB_salidas=miTask[6]
        UB_salidas=miTask[7]

        entradas_ecuacion=miTask[8]
        salidas_ecuacion=miTask[9]
        LB_ecuacion=miTask[10]
        UB_ecuacion=miTask[11]

        
        
        if len(entradas)>0:
            for r in modelo.exchanges:
                r.bounds==[0,0]

            usadasEntradas=set()
            for i in range(len(entradas)):
                nombre=entradas[i]
                compartimento=compartimento_entradas[i]
                if extension=="mat":
                    meta=buscaMetabolitoMat(modelo,nombre,compartimento)
                else:
                    meta=buscaMetabolitoSBML(modelo,nombre,compartimento)
                if compartimento=="[s]":
                    usadasEntradas.add(meta)
                else:
                    inMeta = Reaction("In"+str(i))
                    inMeta.bounds = [LB_entradas[i] ,UB_entradas[i]]
                    inMeta.add_metabolites({
                        meta: -1.0,
                    })
                    modelo.add_reactions([inMeta])

        usadasSalidas=set()
        if len(salidas)>0:
            
            for i in range(len(salidas)):
                nombre=salidas[i]
                compartimento=compartimento_salidas[i]
                if extension=="mat":
                    meta2=buscaMetabolitoMat(modelo,nombre,compartimento)
                else:
                    meta2=buscaMetabolitoSBML(modelo,nombre,compartimento)
                if compartimento=="[s]":
                    usadasSalidas.add(meta2)
                else:
                    outMeta = Reaction("Out"+str(i))
                    outMeta.bounds=[LB_salidas[i],UB_salidas[i]]
                    outMeta.add_metabolites({
                        meta2: -1.0,
                    })
                    modelo.add_reactions([outMeta])
        
        for r in modelo.exchanges:
            m=list(r.metabolites)[0]
            lb=-1000
            ub=1000
            if len(entradas)>0:
                if m in usadasEntradas:
                    lb=-1000
                else:
                    lb=0
            if len(salidas)>0:
                if m in usadasSalidas:
                    ub=1000
                else:
                    ub=0
            r.bounds=[lb,ub]
            if verbose:
                if lb<0 or ub>0:
                    print(r,m.name, lb, ub)
            
        
        if len(entradas_ecuacion)>0 or len(salidas_ecuacion)>0:
            extra = Reaction("Extra")
            extra.bounds = [LB_ecuacion[0],UB_ecuacion[0]]
            for i in range(len(entradas_ecuacion[0])):
                name=entradas_ecuacion[0][i]
                nombre=name[:len(name)-3]
                compartimento=name[len(name)-3:]
                if extension=="mat":
                    meta3=buscaMetabolitoMat(modelo,nombre,compartimento)
                else:
                    meta3=buscaMetabolitoSBML(modelo,nombre,compartimento)
                extra.add_metabolites({
                    meta3: -1.0,
                })
            for i in range(len(salidas_ecuacion[0])):
                name=salidas_ecuacion[0][i]
                nombre=name[:len(name)-3]
                compartimento=name[len(name)-3:]
                if extension=="mat":
                    meta4=buscaMetabolitoMat(modelo,nombre,compartimento)
                else:
                    meta4=buscaMetabolitoSBML(modelo,nombre,compartimento)
                extra.add_metabolites({
                    meta4: 1.0,
                })
            modelo.add_reactions([extra])
        if verbose:
            sol=modelo.optimize()
            if not sol.status=="optimal":
                print("No")
            else:
                if len(entradas_ecuacion)>0 or len(salidas_ecuacion)>0:
                    for j in range(len(modelo.reactions)):
                        r=modelo.reactions[j]
                        if r.id=="Extra":
                            print(sol.fluxes[j],LB_ecuacion[0],UB_ecuacion[0])

        
        else:
            sol=modelo.slim_optimize(error_value=-1.1)
            
            if sol==-1.1:
                print("No")
            else:
                with open(dir2+"/"+name0+"_Task"+str(k+1)+".dat","wb") as f:
                    pickle.dump(modelo,f)
                

In [51]:
def creaModeloTareaHam(modelo,k,verbose):
    with modelo:
        miTask=task[k]
        entradas=miTask[0]
        compartimento_entradas=miTask[1]
        LB_entradas=miTask[2]
        UB_entradas=miTask[3]

        salidas=miTask[4]
        compartimento_salidas=miTask[5]
        LB_salidas=miTask[6]
        UB_salidas=miTask[7]

        entradas_ecuacion=miTask[8]
        salidas_ecuacion=miTask[9]
        LB_ecuacion=miTask[10]
        UB_ecuacion=miTask[11]

        if len(entradas)>0:
            for r in modelo.exchanges:
                r.bounds==[0,0]

            aceptadas=set()
            for i in range(len(entradas)):
                nombre=entradas[i]
                compartimento=compartimento_entradas[i]
                if extension=="mat":
                    meta=buscaMetabolitoMat(modelo,nombre,compartimento)
                else:
                    meta=buscaMetabolitoSBML(modelo,nombre,compartimento)
                aceptadas.add(meta)


            cont=0
            for r in modelo.exchanges:
                m=list(r.metabolites)[0]
                if m in aceptadas:
                    r.bounds=[-1000,1000]
                else:
                    r.bounds=[0,1000]
            
            for r in cobra.util.solver.linear_reaction_coefficients(modelo):
                r.bounds=[1,1000]
            
        if verbose:
            sol=modelo.optimize()
            if not sol.status=="optimal":
                print("No")
            else:
                print("Entradas")
                for i in range(len(entradas)):
                    for j in range(len(modelo.reactions)):
                        r=modelo.reactions[j]
                        if r.id=="In"+str(i):
                            if not r.bounds[0]<=sol.fluxes[j] and r.bounds[1]>=sol.fluxes[j]: 
                                print(i,sol.fluxes[j],r.bounds)
                print("Salidas")
                for i in range(len(salidas)):
                    for j in range(len(modelo.reactions)):
                        r=modelo.reactions[j]
                        if r.id=="Out"+str(i):
                            if not r.bounds[0]<=sol.fluxes[j] and r.bounds[1]>=sol.fluxes[j]: 
                                print(i,sol.fluxes[j],r.bounds)
                if len(entradas_ecuacion)>0 or len(salidas_ecuacion)>0:
                    for j in range(len(modelo.reactions)):
                        r=modelo.reactions[j]
                        if r.id=="Extra":
                            print(sol.fluxes[j],LB_ecuacion[0],UB_ecuacion[0])
            
        else:
            sol=modelo.slim_optimize(error_value=-1.1)
            if sol==-1.1:
                print("No")
            else:
                with open(dir2+"/"+name0+"_Task"+str(k+1)+".dat","wb") as f:
                    pickle.dump(modelo,f)

In [52]:
def creaVariables(tipo):
    global directorio, lista, extension
    if tipo=="base":
        directorio="/home/alumno15/modelos/base/"
        lista=base
        lista=[nombre for nombre in lista if nombre.endswith(".dat")]
        extension=".mat"

    if tipo=="tejidos":
        directorio="/home/alumno15/modelos/tejidos/"
        lista=tejidos
        lista=[nombre for nombre in lista if nombre.endswith(".dat")]
        extension=".xml"

    if tipo=="MM":
        directorio="/home/alumno15/modelos/MM/"
        lista=MM
        lista=[nombre for nombre in lista if nombre.endswith(".dat")]
        extension=".xml"


Change directory to base or tejidos

In [53]:
%%time
tipo="MM"
creaVariables(tipo)

for nombre in lista:
    with open(directorio+nombre,"rb") as f:
        modelo=pickle.load(f)
    name0=nombre[:len(nombre)-4]
    dir2=directorio+name0+"Tasks"
    Path(dir2).mkdir(parents=True, exist_ok=True)

    for i in range(len(task)):
        print()
        print("Task",i)
        if i<len(task)-1:
            try:
                creaModeloTarea(modelo,i,False)
            except:
                print("No puede",i)
        else:
            try:
                creaModeloTareaHam(modelo,i,False)
            except:
                print("No puede",i)


Task 0

Task 1

Task 2

Task 3

Task 4

Task 5

Task 6

Task 7

Task 8

Task 9

Task 10

Task 11

Task 12

Task 13

Task 14

Task 15

Task 16

Task 17

Task 18

Task 19

Task 20

Task 21

Task 22

Task 23

Task 24

Task 25

Task 26

Task 27

Task 28

Task 29

Task 30

Task 31

Task 32

Task 33

Task 34

Task 35

Task 36

Task 37

Task 38

Task 39

Task 40

Task 41

Task 42

Task 43

Task 44

Task 45

Task 46

Task 47

Task 48

Task 49

Task 50

Task 51

Task 52

Task 53

Task 54

Task 55

Task 56

Task 0

Task 1

Task 2

Task 3

Task 4

Task 5

Task 6

Task 7

Task 8

Task 9

Task 10

Task 11

Task 12

Task 13

Task 14

Task 15

Task 16

Task 17

Task 18

Task 19

Task 20

Task 21

Task 22

Task 23

Task 24

Task 25

Task 26

Task 27

Task 28

Task 29

Task 30

Task 31

Task 32

Task 33

Task 34

Task 35

Task 36

Task 37

Task 38

Task 39

Task 40

Task 41

Task 42

Task 43

Task 44

Task 45

Task 46

Task 47

Task 48

Task 49

Task 50

Task 51

Task 52

Task 53

Task 54

Task 55

Ta